# Introduction
In Australia, online shopping is developing fast, the offline shopping is being challenged. It is interesting to know what types of offline shops are popular and welcomed by people. Furthermore, it is also interesting to dig out if there is a different pattern across Australia. 
This will help business stakeholders adapt offline shops together with online shops.

# Business Problem
To understand the the top picked offline shops pattern in Australia, and understand its distribution across different cites of Australia.
# Audience 
* Shopping mall developer, to design/adapt shopping malls, suitable to those popular types of shops
* Shopping mall developer, to understand what types shops are more suitable in different Australian cities
* Shop owners, to open shops attracting more people/values, especially if certain neighbor does not have such type of shops. 
* (Note, this study will not predicate which types shops are missed/demanded in certain location due to limited scope of this study. But of course, this could be one of good extension from the study.)

# Data
This wikipedia link provides a list of shopping centers in Australia. https://en.wikipedia.org/wiki/List_of_shopping_centres_in_Australia
For each shopping center, normally a detailed wikipedia link provided. The detailed page provides coordinates, which can be supplied to Foursquare API.
I have noticed that not all shopping centers have the detailed page. So the study will ignore those shopping centers if the detailed pages not existed. 
Then I plan to use Foursquare API Explore Venues Recommendation, by using extracted latitude and longitude of shopping centers. The API will fetch the top picked shops & category in the shopping center areas. 
# Example
Chadstone Shopping Centre in Victoria, the detailed page (https://en.wikipedia.org/wiki/Chadstone_Shopping_Centre) provides coordinate 37°53′9″S 145°4′57″E. That converts to:
neighborhood_latitude=-38.02
neighborhood_longitude=145.30
Then the url will help to fetch the top picked shops 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)